# 2. Создание нейронной сети без использования готовых решений

__Автор__: Никита Владимирович Блохин (NVBlokhin@fa.ru)

Финансовый университет, 2020 г. 

In [4]:
import torch

## 1. Создание нейронов и полносвязных слоев

1.1. Используя операции над матрицами и векторами из библиотеки `torch`, реализовать нейрон с заданными весами `weights` и `bias`. Прогнать вектор `inputs` через нейрон и вывести результат. 

In [15]:
class Neuron:
    def __init__(self, weights, bias):
        # <создать атрибуты объекта weights и bias>
    
        self.W = weights
        self.B = bias
    
  
    def forward(self, inputs):
        
        # forward path implementation
        
        return torch.matmul(inputs, self.W) + self.B


In [16]:
inputs = torch.tensor([1.0, 2.0, 3.0, 4.0])
weights = torch.tensor([-0.2, 0.3, -0.5, 0.7])
bias = 3.14

# ручная проверка
gt = 1.0*(-0.2)+2.0*0.3 + 3.0*(-0.5)+4.0*0.7 + 3.14

naive_neuron = Neuron(weights, bias)
out = naive_neuron.forward(inputs)
print(out)

# check
assert out == gt

tensor(4.8400)


1.2 Используя операции над матрицами и векторами из библиотеки `torch`, реализовать полносвязный слой с заданными весами `weights` и `biases`. Прогнать вектор `inputs` через слой и вывести результат. 

In [33]:
class Linear:
    def __init__(self, weights, biases):
    # <создать атрибуты объекта weights и biases>
    
        self.W = weights
        self.B = biases
  
    def forward(self, inputs):
        # forward path implementation
        
        return torch.matmul(inputs, self.W) + self.B

In [34]:
inputs = torch.tensor([1.0, 2.0, 3.0, 4.0])
weights = torch.tensor([[-0.2, 0.3, -0.5, 0.7],
                        [0.5, -0.91, 0.26, -0.5],
                        [-0.26, -0.27, 0.17, 0.87]]).T

biases = torch.tensor([3.14, 2.71, 7.2])

naive_neuron = Linear(weights, biases)
out = naive_neuron.forward(inputs)
print(out)

# check
assert out.size()[0] == 3

tensor([ 4.8400,  0.1700, 10.3900])


1.3 Реализовать полносвязный слой из __2.1.2__ таким образом, чтобы он мог принимать на вход матрицу (батч) с данными. Продемонстрировать работу.
Результатом прогона сквозь слой должна быть матрица размера `batch_size` x `n_neurons`.


In [35]:
inputs = torch.tensor([[1, 2, 3, 2.5], 
                       [2, 5, -1, 2], 
                       [-1.5, 2.7, 3.3, -0.8]])
batch_size = 3
n_neurons = 3

out = naive_neuron.forward(inputs)
print(out)

# check
assert out.size()[0] == batch_size and out.size()[1] == n_neurons

tensor([[ 3.7900,  0.9200,  9.0850],
        [ 6.1400, -2.1000,  6.9000],
        [ 2.0400,  0.7610,  6.7260]])


1.4 Используя операции над матрицами и векторами из библиотеки `torch`, реализовать полносвязный слой из `n_neurons` нейронов с `n_features` весами у каждого нейрона (инициализируются из стандартного нормального распределения). Прогнать вектор `inputs` через слой и вывести результат. Результатом прогона сквозь слой должна быть матрица размера `batch_size` x `n_neurons`.

In [64]:
torch.empty(2, 3).normal_(0,1)

tensor([[-0.1705,  0.0199,  0.8357],
        [-0.6815,  1.2642, -1.0900]])

In [76]:
class Linear:
    def __init__(self, n_features, n_neurons):
    # <создать атрибуты объекта weights и biases>
    
        # Создаем случайные веса и смещение нужных размерностей
        self.W = torch.empty(n_neurons, n_features).normal_(0,1)
        self.B = torch.empty(n_neurons).normal_(0,1)    
  
    def forward(self, inputs):
        
        # На этот раз веса необходимо протранспонировать. X * W.T + B
        return torch.matmul(inputs, self.W.T) + self.B
    

In [76]:
inputs = torch.tensor([[1, 2], 
                       [2, 5], 
                       [-1.5, 2.7],
                       [3, 12]])
n_features = inputs.size()[1]
n_neurons = 3
batch_size = inputs.size()[0]

linear_layer = Linear(n_features, n_neurons)
out = linear_layer.forward(inputs)
print(out)

# check
assert out.size()[0] == batch_size and out.size()[1] == n_neurons

tensor([[-0.5754, -1.1635,  4.7333],
        [-2.9272, -2.6839,  7.4405],
        [ 1.5523, -0.3188,  2.4747],
        [-7.1090, -5.6461, 12.3475]])


1.5 Используя решение из __1.4__, создать 2 полносвязных слоя и пропустить матрицу `inputs` последовательно через эти два слоя. Количество нейронов в первом слое выбрать произвольно, количество нейронов во втором слое выбрать так, чтобы результатом прогона являлась матрица (3x7). 

In [80]:
inputs = torch.tensor([[1, 2, 3, 2.5], 
                       [2, 5, -1, 2], 
                       [-1.5, 2.7, 3.3, -0.8]])

n_features = inputs.size()[1]
n_neurons = 5
batch_size = inputs.size()[0]

linear_layer_1 = Linear(n_features, n_neurons)
linear_layer_2 = Linear(n_neurons, 7)

out = linear_layer_2.forward(linear_layer_1.forward(inputs))
print(out)

# check
assert out.size()[0] == 3 and out.size()[1] == 7

tensor([[ -6.5179,  -4.4551,  -3.0809,  -4.3016,  -1.5625,   1.8166,   6.7143],
        [-22.2727,  -5.6442,  -5.4176, -18.6326, -15.6484,   3.3593,  19.6188],
        [-11.9712,  -1.1063,  -4.1587,   1.3513,   4.9698,  -3.7623,  -5.2983]])


Для красоты обернем в класс

In [81]:
import random

In [115]:
class NastiaNN:
    
    def __init__(self, n_layers, n_in_futures= 4, n_out_futures= 7):
        
        # Инициализируем слои нашей сетки
        self.layers = []
        for i in range(n_layers):
            if i == n_layers - 1:
                self.layers.append(Linear(n_neurons, n_out_futures))
                print(f"Создан FC слой формой ({n_in_futures} на {n_out_futures})")
                break
                
            n_neurons = random.randint(2,10)
            self.layers.append(Linear(n_in_futures, n_neurons))
            print(f"Создан FC слой формой ({n_in_futures} на {n_neurons})")
            n_in_futures = n_neurons
            
            
    def forward(self, X):
        out = X
        for layer in self.layers:
            out = layer.forward(out)
        
        return out


In [116]:
network = NastiaNN(5)

network.forward(inputs)

Создан FC слой формой (4 на 9)
Создан FC слой формой (9 на 5)
Создан FC слой формой (5 на 3)
Создан FC слой формой (3 на 6)
Создан FC слой формой (6 на 7)


tensor([[  37.1062,  -47.2284,   83.9885,   -3.5957, -244.5765,   76.6474,
          -38.6390],
        [  -4.6869,  -17.3361,   23.9658,  -40.2316,  -52.6167,   18.6769,
          -42.1530],
        [  44.9802,  -37.4012,   70.4583,   24.1208, -221.5627,   65.3560,
          -10.7416]])

Па бам!

## 2. Создание функций активации

2.1 Используя операции над матрицами и векторами из библиотеки `torch`, реализовать функцию активации ReLU:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/f4353f4e3e484130504049599d2e7b040793e1eb)

Создать матрицу размера (4,3), заполненную числами из стандартного нормального распределения, и проверить работоспособность функции активации.

In [14]:
class ReLU:
    def forward(self, inputs):
        # <реализовать логику ReLU>
        inputs[inputs < 0] = 0
    
        return inputs

In [28]:
inputs = torch.randn((4,3))
print(f'Начальный тензор: \n{inputs}')
relu = ReLU()
print(f'\nТензор после ReLu: \n{relu.forward(inputs)}')


Начальный тензор: 
tensor([[ 0.4534, -0.3830,  2.4398],
        [ 1.2556, -0.0780,  1.7658],
        [-1.0187,  1.1666, -0.6434],
        [ 0.4732, -0.9432, -0.8744]])

Тензор после ReLu: 
tensor([[0.4534, 0.0000, 2.4398],
        [1.2556, 0.0000, 1.7658],
        [0.0000, 1.1666, 0.0000],
        [0.4732, 0.0000, 0.0000]])


2.2 Используя операции над матрицами и векторами из библиотеки `torch`, реализовать функцию активации softmax:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/6d7500d980c313da83e4117da701bf7c8f1982f5)

Создать матрицу размера (4,3), заполненную числами из стандартного нормального распределения, и проверить работоспособность функции активации. Строки матрицы трактовать как выходы линейного слоя некоторого классификатора для 4 различных примеров.

In [57]:
torch.reshape(torch.exp(inputs).sum(1), (-1,1))

tensor([[1.0389],
        [1.6593],
        [2.5922],
        [4.8061]])

In [58]:
class Softmax:
    def forward(self, inputs):
    # <реализовать логику Softmax>
    
        exp_inputs = torch.exp(inputs)
    
        return exp_inputs / torch.reshape(exp_inputs.sum(1), (-1,1))

In [60]:
inputs = torch.randn((4,3))
print(f'Начальный тензор: \n{inputs}')
sm = Softmax()
print(f'\nТензор после Softmax: \n{sm.forward(inputs)}')


Начальный тензор: 
tensor([[ 1.0842, -1.3692, -1.5656],
        [-0.6316,  1.3125,  1.2832],
        [ 0.1270, -4.9263, -2.1252],
        [ 1.5804,  0.9235, -1.5799]])

Тензор после Softmax: 
tensor([[0.8646, 0.0744, 0.0611],
        [0.0677, 0.4730, 0.4593],
        [0.8996, 0.0057, 0.0946],
        [0.6407, 0.3322, 0.0272]])


2.3 Используя операции над матрицами и векторами из библиотеки `torch`, реализовать функцию активации ELU:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/eb23becd37c3602c4838e53f532163279192e4fd)

Создать матрицу размера (4,3), заполненную числами из стандартного нормального распределения, и проверить работоспособность функции активации.

In [71]:
class ELU:
    def __init__(self, alpha):
        # <создать атрибут объекта alpha>
        self.alpha = alpha

    def forward(self, inputs):
        # <реализовать логику ReLU>
        
        alpha_tensor = self.alpha * (torch.exp(inputs) - 1)
        return torch.where(inputs < 0, inputs, alpha_tensor)

In [72]:
inputs = torch.randn((4,3))
print(f'Начальный тензор: \n{inputs}')
elu = ELU(0.5)
print(f'\nТензор после elu: \n{elu.forward(inputs)}')


Начальный тензор: 
tensor([[-0.0215, -0.0943, -1.7014],
        [-1.9720,  0.0455,  0.8348],
        [ 0.7892, -1.1035, -0.7040],
        [ 0.5905,  1.6785, -1.3647]])

Тензор после elu: 
tensor([[-0.0215, -0.0943, -1.7014],
        [-1.9720,  0.0233,  0.6522],
        [ 0.6009, -1.1035, -0.7040],
        [ 0.4025,  2.1787, -1.3647]])


## 3. Создание функции потерь

3.1 Используя операции над матрицами и векторами из библиотеки `torch`, реализовать функцию потерь MSE:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/e258221518869aa1c6561bb75b99476c4734108e)

Создать полносвязный слой с 1 нейроном, прогнать через него батч `inputs` и посчитать значение MSE, трактуя вектор `y` как вектор правильных ответов.

In [73]:
class MSELoss:
    def forward(self, y_pred, y_true):
        
        return ((y_true - y_pred) ** 2).mean()

In [74]:
inputs = torch.tensor([[1, 2, 3, 2.5], 
                       [2, 5, -1, 2], 
                       [-1.5, 2.7, 3.3, -0.8]])

y = torch.tensor([2, 3, 4])

In [81]:
# Создаем Fully connected слой с одним нероном
linear_layer = Linear(4, 1)

mse_loss = MSELoss()
print(f'MSE loss: {mse_loss.forward(linear_layer.forward(inputs), y)}')


MSE loss: 39.20469284057617


3.2 Используя операции над матрицами и векторами из библиотеки `torch`, реализовать функцию потерь Categorical Cross-Entropy:

<img src="https://i.ibb.co/93gy1dN/Screenshot-9.png" width="200">

Создать полносвязный слой с 3 нейронами и прогнать через него батч `inputs`. Полученный результат пропустить через функцию активации softmax. Посчитать значение CCE, трактуя вектор `y` как вектор правильных ответов.

In [89]:
class CategoricalCrossentropyLoss:
    def forward(self, y_pred, y_true):
        # <реализовать логику CCE>
        
        # dim = 1, так как имеем дело с несколькими батчами
        return -1 * (y_true * torch.log(y_pred)).sum(1)

In [83]:
inputs = torch.tensor([[1, 2, 3, 2.5], 
                        [2, 5, -1, 2], 
                        [-1.5, 2.7, 3.3, -0.8]])
y = torch.tensor([1, 0, 0])

In [90]:
# Создаем Fully connected слой с тремя неронами
linear_layer = Linear(4, 3)

softmax = Softmax()
cce_loss = CategoricalCrossentropyLoss()

print(f'CCE loss: {cce_loss.forward(softmax.forward(linear_layer.forward(inputs)), y)}')


CCE loss: tensor([10.0317,  0.3843,  7.0138])


3.3 Модифицировать 2.3.1, добавив L2-регуляризацию.

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/d92ca2429275bfdc0474523babbafe014ca8b580)


In [97]:
class MSELossL:
    
    def __init__(self, lambda_, layer):
        # <создать атрибут объекта alpha>
        self.lambda_ = lambda_
        self.layer = layer

    def data_loss(self, y_pred, y_true):
        # <подсчет первого слагаемого из формулы>
        return ((y_true - y_pred) ** 2).sum()
    
    def reg_loss(self, layer):
        # используйте атрибуты объекта layer, в которых хранятся веса слоя
        # <подсчет второго слагаемого из формулы>
        return self.lambda_ * (layer.W ** 2).sum()
    
    def forward(self, y_pred, y_true):
        return self.data_loss(y_pred, y_true) + self.reg_loss(self.layer)

In [100]:
inputs = torch.tensor([[1, 2, 3, 2.5], 
                        [2, 5, -1, 2], 
                        [-1.5, 2.7, 3.3, -0.8]])
y = torch.tensor([1, 0, 0])

# Создаем Fully connected слой с тремя неронами
linear_layer = Linear(4, 3)
layer_out = linear_layer.forward(inputs)

softmax = Softmax()
mse_loss = MSELossL(1.5, linear_layer)

print(f'MSE loss: {mse_loss.forward(softmax.forward(layer_out), y)}')


MSE loss: 18.79985237121582


## 4. Обратное распространение ошибки

4.1 Используя один нейрон и SGD (1 пример за шаг), решите задачу регрессии

In [107]:
from sklearn.datasets import make_regression

X, y, coef = make_regression(n_features=4, n_informative=4, coef=True, bias=0.5)
X = torch.from_numpy(X).type(torch.float32)# <преобразуйте массивы numpy в тензоры torch с типом torch.float32
y = torch.from_numpy(y).type(torch.float32)# <преобразуйте массивы numpy в тензоры torch с типом torch.float32

[Граф вычислений для этой задачи](https://i.ibb.co/2dhDxZx/photo-2021-02-15-17-18-04.jpg)

4.1.1 Модифицируйте класс `MSELoss` из __2.3.1__, реализовав расчет производной относительно предыдущего слоя


In [108]:
class MSELoss:
    def forward(self, y_pred, y_true):
        return ((y_true - y_pred) ** 2).mean()

    def backward(self, y_pred, y_true):
        # df/dc = 2(с-y) - из графа для вычислений
        self.dinput = 2 * (y_pred - y_true)# df/dc
        return self.dinput

4.1.2. Модифицируйте класс `Neuron` из __2.1.1__:

  1) Сделайте так, чтобы веса нейрона инициализировались из стандартного нормального распределения

  2) Реализуйте расчет градиента относительно весов `weights` и `bias`

In [138]:
class Neuron:
    def __init__(self, n_inputs):
        # <создать атрибуты объекта weights и bias>
        # Создаем случайные веса и смещение нужных размерностей
        self.W = torch.randn(n_inputs)
        self.B = torch.randn(1)    
  
    def forward(self, inputs):
        return torch.matmul(inputs, self.W.T) + self.B
      
    def backward(self, dvalue):
        # dvalue - значение производной, которое приходит нейрону от следующего слоя сети
        # в данном случае это будет значение df/dc (созданное методом backwards у объекта MSELoss)
        self.dweights = dvalue # df/dW - из графа вычислений = 2e*1 = dvalue
        self.dinput =  dvalue * self.W # df/wX = 2e*w = dvalue * W  ----- Зачем здесь dinput?
        self.dbias = dvalue# df/db = 2e = dvalue
        
        # Возвращаем градиент весов и смещения
        return self.dweights, self.dbias
    

In [139]:
inputs = torch.tensor([1.0, 2.0, 3.0, 4.0])

naive_neuron = Neuron(4)
out = naive_neuron.forward(inputs)
print(out)


tensor([2.3527])


4.1.3 Допишите цикл для настройки весов нейрона

[SGD](https://ru.wikipedia.org/wiki/%D0%A1%D1%82%D0%BE%D1%85%D0%B0%D1%81%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B9_%D0%B3%D1%80%D0%B0%D0%B4%D0%B8%D0%B5%D0%BD%D1%82%D0%BD%D1%8B%D0%B9_%D1%81%D0%BF%D1%83%D1%81%D0%BA)

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/dda3670f8a8996a0d3bf80856bb4a166cc8db6d4)

In [136]:
X.size()

torch.Size([100, 4])

In [137]:
y.size()

torch.Size([100])

In [144]:
import numpy as np

In [149]:
n_inputs = X.size(1)# <размерность элемента выборки >
learning_rate = 0.001 #  скорость обучения
n_epoch = 100 #  количество эпох

neuron = Neuron(n_inputs)
loss = MSELoss()

losses = []
for epoch in range(100):
    mean_epoch_loss = []
    # 100 раз выбираем пару X b y
    for x_example, y_example in zip(X, y):
        # forward pass
        y_pred = neuron.forward(x_example)# <прогон через нейрон>
        curr_loss = loss.forward(y_pred, y_example)# <прогон через функцию потерь>
        losses.append(curr_loss)
        mean_epoch_loss.append(curr_loss)
        
        # backprop
        # <вызов методов backward>
        # обратите внимание на последовательность вызовов: от конца к началу
        
        # Получаем градиенты весов и смещений
        dW, dB = neuron.backward( loss.backward(y_pred, y_example) )
        
        # <шаг оптимизации для весов (weights и bias) нейрона>
        neuron.W -= learning_rate * dW
        neuron.B -= learning_rate * dB
        
    print(f"Epoch {epoch}, Average loss= {np.array(mean_epoch_loss).mean()}")

Epoch 0, Average loss= 11027.037109375
Epoch 1, Average loss= 10654.146484375
Epoch 2, Average loss= 10362.1376953125
Epoch 3, Average loss= 10132.431640625
Epoch 4, Average loss= 9951.0478515625
Epoch 5, Average loss= 9807.369140625
Epoch 6, Average loss= 9693.259765625
Epoch 7, Average loss= 9602.4404296875
Epoch 8, Average loss= 9530.0322265625
Epoch 9, Average loss= 9472.21875
Epoch 10, Average loss= 9426.00390625
Epoch 11, Average loss= 9389.02734375
Epoch 12, Average loss= 9359.4189453125
Epoch 13, Average loss= 9335.6962890625
Epoch 14, Average loss= 9316.6796875
Epoch 15, Average loss= 9301.4287109375
Epoch 16, Average loss= 9289.1953125
Epoch 17, Average loss= 9279.3779296875
Epoch 18, Average loss= 9271.5009765625
Epoch 19, Average loss= 9265.17578125
Epoch 20, Average loss= 9260.099609375
Epoch 21, Average loss= 9256.0224609375
Epoch 22, Average loss= 9252.75
Epoch 23, Average loss= 9250.1220703125
Epoch 24, Average loss= 9248.009765625
Epoch 25, Average loss= 9246.314453125

In [150]:
# Поигравшись с lr становится понятно, что нейрон просто обчень быстро скатывается в локальный минимум.
# В данном случае, после 37 похи loss перестает уменьшаться

4.2 Решите задачу 2.4.1, используя пакетный градиентный спуск

Вычисления для этой задачи: 
[1](https://i.ibb.co/rmtQT6P/photo-2021-02-15-18-00-43.jpg)
[2](https://i.ibb.co/NmCFVnQ/photo-2021-02-15-18-01-17.jpg)

# ???

In [158]:
n_inputs = X.size(1)# <размерность элемента выборки >
learning_rate = 0.1 #  скорость обучения
n_epoch = 100 #  количество эпох

neuron = Neuron(n_inputs)
loss = MSELoss()
optimizer = torch.optim.SGD([neuron.W, neuron.B], lr=learning_rate)

losses = []
for epoch in range(100):
    mean_epoch_loss = []
    # 100 раз выбираем пару X b y
    for x_example, y_example in zip(X, y):
        # forward pass
        y_pred = neuron.forward(x_example)# <прогон через нейрон>
        curr_loss = loss.forward(y_pred, y_example)# <прогон через функцию потерь>
        losses.append(curr_loss)
        mean_epoch_loss.append(curr_loss)
        
        loss.backward(y_pred, y_example)          
        
        optimizer.step()
        optimizer.zero_grad()
        
    print(f"Epoch {epoch}, Average loss= {np.array(mean_epoch_loss).mean()}")

Epoch 0, Average loss= 11029.3935546875
Epoch 1, Average loss= 11029.3935546875
Epoch 2, Average loss= 11029.3935546875
Epoch 3, Average loss= 11029.3935546875
Epoch 4, Average loss= 11029.3935546875
Epoch 5, Average loss= 11029.3935546875
Epoch 6, Average loss= 11029.3935546875
Epoch 7, Average loss= 11029.3935546875
Epoch 8, Average loss= 11029.3935546875
Epoch 9, Average loss= 11029.3935546875
Epoch 10, Average loss= 11029.3935546875
Epoch 11, Average loss= 11029.3935546875
Epoch 12, Average loss= 11029.3935546875
Epoch 13, Average loss= 11029.3935546875
Epoch 14, Average loss= 11029.3935546875
Epoch 15, Average loss= 11029.3935546875
Epoch 16, Average loss= 11029.3935546875
Epoch 17, Average loss= 11029.3935546875
Epoch 18, Average loss= 11029.3935546875
Epoch 19, Average loss= 11029.3935546875
Epoch 20, Average loss= 11029.3935546875
Epoch 21, Average loss= 11029.3935546875
Epoch 22, Average loss= 11029.3935546875
Epoch 23, Average loss= 11029.3935546875
Epoch 24, Average loss= 11

4.2.1 Модифицируйте класс `MSELoss` из __3.1__, реализовав расчет производной относительно предыдущего слоя с учетом того, что теперь работа ведется с батчами, а не с индивидуальными примерами
 

In [ ]:
class MSELoss:
  def forward(self, y_pred, y_true):
    return # <реализовать логику MSE>

  def backward(self, y_pred, y_true):
    self.dinput = # df/dy^


4.2.2. Модифицируйте класс `Neuron` из __4.1.2__:

  1) Реализуйте метод `forward` таким образом, чтобы он мог принимать на вход матрицу (батч) с данными. 

  2) Реализуйте расчет градиента относительно весов `weights` и `bias` с учетом того, что теперь работа ведется с батчами, а не с индивидуальными примерами

In [ ]:
class Neuron:
  def __init__(self, n_inputs):
    # <создать атрибуты объекта weights и bias>
    pass
  
  def forward(self, inputs):
    return # <реализовать логику нейрона>
  
  def backward(self, dvalue):
    # dvalue - значение градиента, которое приходит нейрону от следующего слоя сети
    # в данном случае это будет градиент L по y^ (созданный методом backwards у объекта MSELoss)
    self.dweights = # df/dW
    self.dbias = # df/db


4.2.3 Допишите цикл для настройки весов нейрона

In [ ]:
n_inputs = # <размерность элемента выборки >
learning_rate = 0.1 #  скорость обучения
n_epoch = 100 #  количество эпох

neuron = Neuron(n_inputs)
loss = MSELoss()


for epoch in range(100):
    # forward pass
    y_pred = # <прогон через нейрон>
    curr_loss = # <прогон через функцию потерь>
    losses.append(curr_loss)

    # backprop
    # <вызов методов backward>
    # обратите внимание на последовательность вызовов: от конца к началу

    # <шаг оптимизации для весов (weights и bias) нейрона>

4.3  Используя один полносвязный слой и  пакетный градиетный спуск, решите задачу регрессии из __2.4.1__

4.3.1 Модифицируйте класс `Linear` из __1.4__. ([вычисление градиентов](https://i.ibb.co/kgVR6m6/photo-2021-02-15-21-30-28.jpg))

In [ ]:
class Linear:
  def __init__(self, n_features, n_neurons):
    # <создать атрибуты объекта weights и biases>
    pass
  
  def forward(self, inputs):
    return # <реализовать логику слоя>

  def backward(self, dvalues):
    self.dweights = # df/dW
    self.dbiases = # df/db
    self.dinputs = # df/dX

4.3.2 Создайте слой с одним нейроном. Используя класс MSELoss из 2.4.2, убедитесь, что модель обучается

4.4 Используя наработки из 2.4, создайте нейросеть и решите задачу регрессии.

Предлагаемая архитектура: 
1. Полносвязный слой с 10 нейронами
2. Активация ReLU
3. Полносвязный слой с 1 нейроном

In [ ]:
X = torch.linspace(-1, 1, 100).view(-1, 1)
y = X.pow(2) + 0.2 * torch.rand(X.size()) 

In [ ]:
class Activation_ReLU:
  def forward(self, inputs):
    self.inputs = inputs
    self.output = inputs.clip(min=0)
    return self.output
  
  def backward(self, dvalues):
    self.dinputs = dvalues.clone()
    self.dinputs[self.inputs <= 0] = 0

In [ ]:
# создание компонентов сети
# fc1 = 
# relu1 = 
# fc2 = 

loss = MSELoss()
lr = 0.02

ys = []
for epoch in range(2001):
  # <forward pass>
  # fc1 > relu1 > fc2 > loss

  data_loss = # <прогон через функцию потерь>

  if epoch % 200 == 0:
    print(f'epoch {epoch} mean loss {data_loss}')
    ys.append(out)
  
  # <backprop> 
  # loss > fc2 > relu1 > fc1

  # <шаг оптимизации для fc1>

  # <шаг оптимизации для fc2>


In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(len(ys), 1, figsize=(10, 40))
for ax, y_ in zip(axs, ys):
  ax.scatter(X.numpy(), y.numpy(), color = "orange")
  ax.plot(X.numpy(), y_.numpy(), 'g-', lw=3)
  ax.set_xlim(-1.05, 1.5)
  ax.set_ylim(-0.25, 1.25)